In [26]:
from tifffile import imread, imwrite
import napari
from skimage import measure

In [4]:
labels = imread('./training_data/thick_brain_Nikon_dagnysd_0.5/y(512x512)/X_15.tif')

In [5]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_labels(labels)

Assistant skips harvesting pyclesperanto as it's not installed.


<Labels layer 'labels' at 0x23b1fa1cf70>

In [6]:
labels.shape

(15, 512, 512)

In [11]:
import numpy as np

In [12]:
roi = viewer.layers[1].data
# Perform maximum intensity projection (MIP) from the label stack
label_mip = np.max(roi, axis=0)

# We will create a mask where label_mip is greater than or equal to 1
mask = (label_mip >= 1).astype(np.uint8)

In [13]:
labels_2D = labels[0,:,:]

In [14]:
labels_2D.shape

(512, 512)

In [15]:
viewer.add_labels(labels_2D)

<Labels layer 'labels_2D' at 0x23b1899f2b0>

In [16]:
viewer.add_image(mask)

<Image layer 'mask' at 0x23b2e910b80>

In [20]:
# Save mask (binary image)
imwrite("./test_mask.tiff", mask)

In [21]:
# Load roi from tiff
roi = imread("./test_mask.tiff")

In [22]:
viewer.add_labels(roi)

<Labels layer 'roi' at 0x23b30185e80>

In [31]:
def extract_contour(roi: np.ndarray) -> np.ndarray:
    """
    Extracts the contour of a binary ROI image and returns a binary mask 
    where the contour pixels are set to 1.

    Parameters:
    -----------
    roi : np.ndarray
        A 2D NumPy array of dtype int8 representing the binary region of interest (ROI),
        where nonzero values indicate the foreground.

    Returns:
    --------
    np.ndarray
        A binary mask of the same shape as `roi`, where contour pixels are set to 1,
        and all other pixels are 0.

    Notes:
    ------
    - Uses `skimage.measure.find_contours` to detect continuous contour points.
    - Vectorized operations are used for efficiency.
    - The function assumes that `roi` is a binary image (values of 0 or 1).

    Example:
    --------
    >>> import numpy as np
    >>> from skimage.draw import disk
    >>> roi = np.zeros((100, 100), dtype=np.int8)
    >>> rr, cc = disk((50, 50), 20)
    >>> roi[rr, cc] = 1
    >>> contour_mask = extract_contour(roi)
    >>> print(contour_mask.sum())  # Nonzero values represent contour pixels
    """
    # Find contours, output is a list of (N, 2) arrays representing continuous (x, y) coordinates of contour points
    contours = measure.find_contours(roi, level=0.5)

    # Concatenate all contour points
    all_contours = np.vstack(contours)  # Shape: (N, 2)

    # Round to integer pixel indices
    all_contours = np.round(all_contours).astype(int)

    # Clip to ensure indices are within image bounds
    all_contours[:, 0] = np.clip(all_contours[:, 0], 0, roi.shape[0] - 1)
    all_contours[:, 1] = np.clip(all_contours[:, 1], 0, roi.shape[1] - 1)

    # Create an empty binary mask
    contour_mask = np.zeros_like(roi, dtype=np.uint8)

    # Set pixels at contour locations to 1 using NumPy advanced indexing
    contour_mask[all_contours[:, 0], all_contours[:, 1]] = 1
    
    return contour_mask


In [32]:
contour_mask = extract_contour(roi)

In [33]:
viewer.add_labels(contour_mask)

<Labels layer 'contour_mask' at 0x23b2cffd9d0>